In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from copy import deepcopy

from symmetries.analysis.lie_symmetry_analysis import point_symmetries
from symmetries.utils.symbolic import sym_det_eqn

from symmetries.objects.determining_equations import DeterminingEquations
import sympy as sp
from sympy import Derivative as D

# Function to propose a general form of the infinitesimals.

### Continuity Equation

In [77]:
x, t = sp.symbols('x t')
rho = sp.Function('rho')(x, t)
v = sp.Function('v')(x, t)
independent_variables = [x,t]
dependent_variables = [v, rho]
constants = []
F =  D(rho,t) + rho*D(v,x)
F_rules_array = {D(rho,t):-rho*D(v,x)}
order = 1
F.expand()

rho(x, t)*Derivative(v(x, t), x) + Derivative(rho(x, t), t)

In [78]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [79]:
system_of_equations.print_determining_equations()

Matrix([
[0, Eq(eta^(\rho) - eta^(\rho)_rho*rho(x, t) + eta^(v)_v*rho(x, t), 0)],
[1,                          Eq(eta^(\rho)_t + eta^(v)_x*rho(x, t), 0)],
[2,                                                  Eq(xi^(t)_rho, 0)],
[3,                                                      Eq(xi^(x), 0)],
[4,                                                 Eq(eta^(v)_rho, 0)],
[5,                                                      Eq(xi^(t), 0)],
[6,                                                Eq(eta^(\rho)_v, 0)]])

In [89]:
system_of_equations.print_latex()

1\eta^{\rho(x, t)}-1\rho\eta^{\rho(x, t)_{\rho}+1\rho\eta^{v(x, t)}_{v}=0\\
1\rho\eta^{v(x, t)}_{x}+1\eta^{\rho(x, t)_{t}=0\\
\xi^{t}_{\rho}=0\\
\xi^{x}}=0\\
\eta^{v(x, t)}_{\rho}=0\\
\xi^{t}}=0\\
\eta^{\rho(x, t)_{v}=0\\



In [ ]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
independent_variables = [x, t]
dependent_variables = [u]
constants = [k]
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [ ]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

deleting xit u(x, t)
deleting xix u(x, t)
deleting xit x
found deleted variable in xix eq 3
found deleted variable in xix eq 3
found high order derivative of deleted variable in xit eq 0
found high order derivative of deleted variable in xix eq 1


In [ ]:
system_of_equations.print_determining_equations()

Matrix([
[1,                                   Eq(xi^(t)_u, 0)],
[2,                                Eq(eta^(u)_u_u, 0)],
[3,                                   Eq(xi^(x)_u, 0)],
[4,                                   Eq(xi^(t)_x, 0)],
[5,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[6, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[7,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [ ]:
a = np.array([0,0,0,0,1])

In [ ]:
len(np.where(a!=0))==1

True

In [ ]:
g = deepcopy(general_form.general_form)
# dict(np.arange(0,len(variables), (variables)))
# hdptm[2] -> x
for eqn in general_form.determining_equations:
    for infinitesimal in g.keys():
        for variable in infinitesimal.keys():
            infinitesimal_variable = f'{infinitesimal}{variable}'
            general_form.determining_equations[3]
            if len(eqn)==1 and 


SyntaxError: invalid syntax (2066456965.py, line 3)

# Fix Torres' issues so he is not mad anymore :c

In [ ]:
import sympy as sp
import numpy as np
import re
import ipdb
from utils.algebra import str_eqn_to_dict_eqn

In [ ]:
x = sp.symbols('x')
y = sp.Function('y')(x)
list_indep = [x]
list_dep = [y]
list_cte = []

list_var = list_indep + list_dep
list_all = list_cte + list_indep + list_dep

In [ ]:
def parens(s):
    i = s.count(')') - 1
    groups = s[s.find('('):].split(')')
    return (')'.join(groups[:i]) + ')')

In [ ]:
def str_to_dict(f, term, arr_pow, arr_deriv, list_all, list_var):
    if f.is_Mul:
        for i in f.args:
            str_to_dict(i, term, arr_pow, arr_deriv, list_all, list_var)
    else:
        if f.args == ():
            if f.is_Integer:
                term['coefficient'] = f
            else:
                idx = np.where(list_all == f)
                p = 1
                arr_pow[idx] = p

        if f.is_Function:
            idx = np.where(list_all == f)
            p = 1
            arr_pow[idx] = p
            if idx[0].size == 0:
                term['variable'] = str(f).split('(')[0]

        if f.is_Pow:
            var = f.args[0]
            if var.is_Derivative:
                term['variable'] = var.args[0]
                for v in var.args[1:]:
                    idx = np.where(list_var == v[0])
                    arr_deriv[idx] = v[1]
            else:
                idx = np.where(list_all == f.args[0])
                arr_pow[idx] = f.args[1]

        if type(f) == type(sp.Subs(list_var[0],
                                   list_var[0], list_var[0])):
            s = str(f)
            if s.endswith('))'):
                if '(' in re.split(',', s)[-1]:
                    subs = re.split(',',s)[-2].strip(' ')
                    subs_t = re.split(',',s)[-1].strip(')').strip(' ')    # Changed
                    subs_t = subs_t + ')'
                else:
                    subs = re.split(',', s)[-3].strip(' ')
                    subs_t = re.split(',', s)[-2] + ',' + re.split(',', s)[-1]
                    subs_t = subs_t[:-1].strip(' ')
            else:
                subs = re.split(',',s)[-2].strip(' ')
                subs_t = re.split(',',s)[-1].strip(')').strip(' ')
            if ')' in subs:
                subs = subs.strip(')')
            s = s.replace(subs, subs_t)
            f = sp.sympify(parens(s).strip('('))
            if isinstance(f, tuple):
                f = f[0]

        if f.is_Derivative:
            term['variable'] = str(f.args[0]).split("(")[0]
            for v in f.args[1:]:
                idx = np.where(list_var == v[0])
                arr_deriv[idx] = v[1]

    term['constants'] = list(arr_pow.astype(int))
    term['derivatives'] = list(arr_deriv.astype(int))
    return term


In [ ]:
dict_det_eqn = {'Derivative(y(x),x)*Derivative(y(x),(x,2))': ['-xix(x,y(x))*y(x)',
  '+3*y(x)^2*Derivative(xix(x,y(x)),y(x))'],
 'Derivative(y(x),(x,4))': ['-xix(x,y(x))*y(x)'],
 'Derivative(y(x),x)*Derivative(y(x),(x,3))': ['-2*xix(x,y(x))'],
 'Derivative(y(x),(x,2))^2': ['-xix(x,y(x))'],
 'Derivative(y(x),x)^2': ['+y(x)*Derivative(etay(x,y(x)),(y(x),2))',
  '-y(x)*Subs(Derivative(xix(_xi_1,y(x)),_xi_1,y(x)),_xi_1,x)',
  '-y(x)*Subs(Derivative(xix(x,_xi),_xi,x),_xi,y(x))'],
 'Derivative(y(x),x)^3': ['-y(x)*Derivative(xix(x,y(x)),(y(x),2))'],
 'Derivative(y(x),x)': ['-y(x)*Subs(Derivative(xix(_xi_1,y(x)),(_xi_1,2)),_xi_1,x)',
  '+y(x)*Subs(Derivative(etay(_xi_1,y(x)),_xi_1,y(x)),_xi_1,x)',
  '+y(x)*Subs(Derivative(etay(x,_xi),_xi,x),_xi,y(x))'],
 'Derivative(y(x),(x,2))': ['-etay(x,y(x))*y(x)',
  '-y(x)^2*Derivative(etay(x,y(x)),y(x))',
  '+2*y(x)^2*Subs(Derivative(xix(_xi_1,y(x)),_xi_1),_xi_1,x)'],
 'lonely_terms': ['+y(x)*Subs(Derivative(etay(_xi_1,y(x)),(_xi_1,2)),_xi_1,x)']}

In [ ]:
det_eqn = []
for eqn in dict_det_eqn.values():
    aux_list = []
    for str_term in eqn:
        arr_pow = np.zeros(len(list_all))
        arr_deriv = np.zeros(len(list_var))
        term = {"coefficient": 1, "constants": None,
                "derivatives": None, "variable": None}
        aux_list.append(str_to_dict(sp.sympify(str_term), term, arr_pow,
              arr_deriv, np.array(list_all), np.array(list_var)))
    det_eqn.append(aux_list)
keys = list(np.arange(len(det_eqn)))
ret = dict(zip(keys, det_eqn))

Subs(Derivative(xix(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x)
Subs(Derivative(xix(x, _xi), _xi, x), _xi, y(x))
Subs(Derivative(xix(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)
Subs(Derivative(etay(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x)
Subs(Derivative(etay(x, _xi), _xi, x), _xi, y(x))
Subs(Derivative(xix(_xi_1, y(x)), _xi_1), _xi_1, x)
Subs(Derivative(etay(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)


In [ ]:
ret

{0: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 0],
   'variable': 'xix'},
  {'coefficient': 3,
   'constants': [0, 2],
   'derivatives': [0, 1],
   'variable': 'xix'}],
 1: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 2: [{'coefficient': -2,
   'constants': [0, 0],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 3: [{'coefficient': -1,
   'constants': [0, 0],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 4: [{'coefficient': 1,
   'constants': [0, 1],
   'derivatives': [0, 2],
   'variable': 'etay'},
  {'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [1, 1],
   'variable': 'xix'},
  {'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [1, 1],
   'variable': 'xix'}],
 5: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 2],
   'variable': 'xix'}],
 6: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [2, 0],
   'variable': 'xix'},
  {'coefficient': 1,
   